# Comment générer du texte : utilisation de différentes méthodes de décodage pour la génération de langage avec Transformers

Credits: https://huggingface.co/blog/how-to-generate

### **Introduction**

Ces dernières années, la génération de langage ouvert a suscité un intérêt croissant grâce à l'essor de grands modèles de langage basés sur des transformateurs et entraînés sur des millions de pages web, tels que le célèbre [modèle GPT2] d'OpenAI (https://openai.com/blog/better-language-models/). Les résultats de la génération conditionnée de langage ouvert sont impressionnants, par exemple [GPT2 sur les licornes](https://openai.com/blog/better-language-models/#samples), [XLNet](https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e), [Langage contrôlé avec CTRL](https://blog.einstein.ai/introducing-a-conditional-transformer-language-model-for-controllable-generation/). Outre l'architecture améliorée du transformateur et les données d'entraînement massives non supervisées, **de meilleures méthodes de décodage** ont également joué un rôle important.

Cet article de blog donne un bref aperçu des différentes stratégies de décodage et, plus important encore, montre comment *vous* pouvez les mettre en œuvre avec très peu d'effort en utilisant la bibliothèque populaire `transformers` !

Toutes les fonctionnalités suivantes peuvent être utilisées pour la génération **auto-régressive** de langage ([ici](http://jalammar.github.io/illustrated-gpt2/) un rappel). En bref, la génération *auto-régressive* de langage est basée sur l'hypothèse que la distribution de probabilité d'une séquence de mots peut être décomposée en un produit de distributions conditionnelles des mots suivants :
$$ P(w_{1:T} | W_0 ) = \prod_{t=1}^T P(w_{t} | w_{1 : t-1}, W_0) \text{ ,avec } w_{1 : 0} = \emptyset, $$

et $W_0$ étant la séquence de mots *contexte* initiale. La longueur $T$ de la séquence de mots est habituellement déterminée *à la volée* et correspond au pas de temps $t=T$ où le jeton EOS est généré à partir de $P(w_{t} | w_{1 : t-1}, W_{0})$.


La génération de langage auto-régressive est maintenant disponible pour `GPT2`, `XLNet`, `OpenAi-GPT`, `CTRL`, `TransfoXL`, `XLM`, `Bart`, `T5` dans PyTorch et Tensorflow >= 2.0 !

Nous ferons un tour d'horizon des méthodes de décodage les plus courantes, principalement *Greedy search*, *Beam search*, *Top-K sampling* et *Top-p sampling*.


Installons rapidement les transformateurs et chargeons le modèle. Nous utiliserons GPT2 dans Tensorflow 2.1 pour la démonstration, mais l'API est 1-à-1 la même pour PyTorch.

In [ ]:
!pip install -q transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)

### **Recherche avide**

La recherche avide sélectionne simplement le mot ayant la probabilité la plus élevée comme mot suivant : $w_t = argmax_{w}P(w | w_{1:t-1})$ à chaque étape $t$. L'esquisse suivante illustre la recherche avide.

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

En partant du mot $\text{"The"}$, l'algorithme
choisit avec avidité le mot suivant ayant la probabilité la plus élevée $\text{"nice"}$ et ainsi de suite, de sorte que la séquence de mots finale générée est $\text{"The", "nice", "woman"}$ avec une probabilité globale de 0,5 \times 0,4 = 0,2$.

Dans ce qui suit, nous allons générer des séquences de mots en utilisant GPT2 sur le contexte $(\text{"I", "enjoy", "walking", "with", "my", "cute", "dog"})$. Voyons comment la recherche avide peut être utilisée dans les "transformateurs" comme suit :

In [ ]:
# encode context the generation is conditioned on
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

# generate 40 new tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=40)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure


Très bien ! Nous avons généré notre premier texte court avec GPT2 😊. Les mots générés suivant le contexte sont raisonnables, mais le modèle commence rapidement à se répéter ! Il s'agit d'un problème très courant dans la génération de langage en général et qui semble l'être encore plus dans la recherche avide et la recherche par faisceau - voir [Vijayakumar et al., 2016](https://arxiv.org/abs/1610.02424) et [Shao et al., 2017](https://arxiv.org/abs/1701.03185).

L'inconvénient majeur de la recherche avide est cependant qu'elle manque des mots à forte probabilité cachés derrière un mot à faible probabilité, comme on peut le voir dans notre croquis ci-dessus :

Le mot $\text{"has"}$ avec sa probabilité conditionnelle élevée de 0,9$ est caché derrière le mot $\text{"dog"}$, qui n'a que la deuxième probabilité conditionnelle la plus élevée, de sorte que la recherche avide manque la séquence de mots $\text{"The"}, \text{"dog"}, \text{"has"}$.

Heureusement, la recherche par faisceau permet de résoudre ce problème !

### **Recherche par faisceau**

La recherche par faisceau réduit le risque de manquer des séquences de mots cachés à forte probabilité en conservant le `nombre de faisceaux` d'hypothèses le plus probable à chaque pas de temps et en choisissant finalement l'hypothèse qui a la probabilité globale la plus élevée. Prenons un exemple avec `num_beams=2` :

![Recherche de faisceaux](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

Au pas de temps $1$, en plus de l'hypothèse la plus probable $\text{"The", "nice"}$, la recherche par faisceau garde également la trace de la deuxième hypothèse la plus probable $\text{"The", "dog"}$. Au pas de temps $2$, beam search constate que la séquence de mots $\text{"The", "dog", "has"}$ a une probabilité supérieure de $0.36$ à $\text{"The", "nice", "woman"}$, qui a une probabilité de $0.2$. Génial, il a trouvé la séquence de mots la plus probable dans notre exemple de jouet !

La recherche par faisceau trouvera toujours une séquence de sortie avec une probabilité plus élevée que la recherche avide, mais il n'est pas garanti qu'elle trouve la sortie la plus probable.

Voyons comment la recherche par faisceau peut être utilisée dans `transformers`. Nous mettons `num_beams > 1` et `early_stopping=True` pour que la génération soit terminée lorsque toutes les hypothèses de faisceau ont atteint le jeton EOS.

In [ ]:
# activate beam search and early_stopping
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure


Bien que le résultat soit sans doute plus fluide, la sortie comprend toujours des répétitions des mêmes séquences de mots.  
Un remède simple consiste à introduire des pénalités *n-grammes* (*a.k.a* séquences de $n$ mots) comme introduit par [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304) et [Klein et al. (2017)](https://arxiv.org/abs/1701.02810). La pénalité *n-grams* la plus courante permet de s'assurer qu'aucun *n-gramme* n'apparaît deux fois en fixant manuellement la probabilité des mots suivants susceptibles de créer un *n-gramme* déjà vu à $0$.

Essayons-le en réglant `no_repeat_ngram_size=2` pour qu'aucun *2-gramme* n'apparaisse deux fois :

In [ ]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to


Joli, c'est beaucoup mieux ! Nous pouvons voir que la répétition n'apparaît plus. Néanmoins, les pénalités *n-gram* doivent être utilisées avec précaution. Un article généré sur la ville *New York* ne doit pas utiliser une pénalité *2-grammes*, sinon le nom de la ville n'apparaîtrait qu'une seule fois dans l'ensemble du texte !

Une autre caractéristique importante de la recherche par faisceau est que nous pouvons comparer les meilleurs faisceaux après la génération et choisir le faisceau généré qui correspond le mieux à notre objectif.

Dans `transformers`, nous fixons simplement le paramètre `num_return_sequences` au nombre de faisceaux les mieux notés qui doivent être retournés. Assurez-vous cependant que `num_return_sequences <= num_beams` !

In [ ]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to
1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to
2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea to
3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time to take a
4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea

Comme on peut le voir, les cinq hypothèses de faisceaux ne sont que marginalement différentes les unes des autres - ce qui ne devrait pas être trop surprenant lorsqu'on n'utilise que 5 faisceaux.

En ce qui concerne la génération ouverte, quelques raisons ont récemment été avancées pour expliquer pourquoi la recherche par faisceau n'est peut-être pas la meilleure option possible :

- La recherche par faisceau peut très bien fonctionner dans des tâches où la longueur de la génération souhaitée est plus ou moins prévisible, comme dans la traduction automatique ou le résumé - voir [Murray et al. (2018)](https://arxiv.org/abs/1808.10006) et [Yang et al. (2018)](https://arxiv.org/abs/1808.09582). Mais ce n'est pas le cas pour la génération ouverte où la longueur de sortie souhaitée peut varier considérablement, par exemple la génération de dialogues et d'histoires.

- Nous avons vu que la recherche par faisceau souffre fortement de la génération répétitive. Ceci est particulièrement difficile à contrôler avec des pénalités *n-gram*- ou autres dans la génération d'histoires puisque trouver un bon compromis entre la "non-répétition" forcée et la répétition de cycles de *n-grammes* identiques demande beaucoup d'ajustements.

- Comme indiqué dans [Ari Holtzman et al. (2019)] (https://arxiv.org/abs/1904.09751), un langage humain de haute qualité ne suit pas une distribution de mots suivants à forte probabilité. En d'autres termes, en tant qu'humains, nous voulons que le texte généré nous surprenne et ne soit pas ennuyeux/prévisible. Les auteurs le montrent joliment en traçant la probabilité qu'un modèle donnerait à un texte humain par rapport à ce que fait la recherche par faisceau.

![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)


Alors arrêtons d'être ennuyeux et introduisons un peu de hasard 🤪.

### **Échantillonnage**

Dans sa forme la plus basique, l'échantillonnage consiste à choisir aléatoirement le mot suivant $w_t$ en fonction de sa distribution de probabilité conditionnelle :

$$w_t \sim P(w|w_{1:t-1})$$

En reprenant l'exemple ci-dessus, le graphique suivant visualise la génération du langage lors de l'échantillonnage.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

Il est évident que la génération de langage par échantillonnage n'est plus *déterministe*. Le mot
est échantillonné à partir de la distribution de probabilité conditionnée $P(w | \text{"The"})$, suivi par l'échantillonnage de $\text{"drives"}$ à partir de $P(w | \text{"The"}, \text{"car"})$.

Dans `transformers`, nous fixons `do_sample=True` et désactivons l'échantillonnage *Top-K* (nous y reviendrons) via `top_k=0`. Dans ce qui suit, nous fixerons `random_seed=0` à des fins d'illustration. N'hésitez pas à modifier `random_seed` pour jouer avec le modèle.

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
from transformers import set_seed
set_seed(42)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog for the rest of the day, but this had me staying in an unusual room and not going on nights out with friends (which will always be wondered for a mere minute or so at this point).


Intéressant! Le texte semble correct, mais en y regardant de plus près, il n'est pas très cohérent. les *3 grammes* *le nouveau sens de la main* et le *harnais de frappe local* sont très étranges et ne semblent pas avoir été écrits par un humain. C'est le gros problème lors de l'échantillonnage de séquences de mots : les modèles génèrent souvent un charabia incohérent, *cf.* [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751).

Une astuce consiste à rendre la distribution $P(w|w_{1:t-1})$ plus nette (augmentant la probabilité de mots à forte probabilité et diminuant la probabilité de mots à faible probabilité) en abaissant la « température » de le [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max).

Une illustration de l’application de la température à notre exemple ci-dessus pourrait ressembler à ceci.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

La distribution conditionnelle du mot suivant de l'étape $t=1$ devient beaucoup plus nette, ne laissant presque aucune chance au mot $\text{"car"}$ d'être sélectionné.


Voyons comment nous pouvons refroidir la distribution dans la bibliothèque en définissant `temperature=0.7` :

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0,
    temperature=0.6,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I don't like to chew on it. I like to eat it and not chew on it. I like to be able to walk with my dog."

So how did you decide


D'ACCORD. Il y a moins de n-grammes bizarres et le résultat est un peu plus cohérent maintenant ! Bien que l'application de la température puisse rendre une distribution moins aléatoire, dans sa limite, lors du réglage de la « température » $ \à 0$, l'échantillonnage échelonné en température devient égal à un décodage glouton et souffrira des mêmes problèmes qu'auparavant.

### **Échantillonnage Top-K**

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf) a introduit un schéma d'échantillonnage simple mais très puissant, appelé échantillonnage ***Top-K***. Dans l'échantillonnage *Top-K*, les *K* mots suivants les plus probables sont filtrés et la masse de probabilité est redistribuée uniquement parmi ces *K* mots suivants.
GPT2 a adopté ce schéma d'échantillonnage, ce qui a été l'une des raisons de son succès dans la génération d'histoires.

Nous étendons la gamme de mots utilisés pour les deux étapes d'échantillonnage dans l'exemple ci-dessus de 3 mots à 10 mots pour mieux illustrer l'échantillonnage *Top-K*.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

Après avoir défini $K = 6$, dans les deux étapes d'échantillonnage, nous limitons notre pool d'échantillonnage à 6 mots. Alors que les 6 mots les plus probables, définis comme $V_{\text{top-K}}$, n'englobent que *environ.* les deux tiers de la masse de probabilité totale dans la première étape, ils incluent la quasi-totalité de la masse de probabilité dans la première étape. deuxième étape. Néanmoins, on voit qu'il réussit à éliminer les candidats plutôt bizarres $\text{"not", "the", "small", "told"}$
lors de la deuxième étape d’échantillonnage.


Voyons comment *Top-K* peut être utilisé dans la bibliothèque en définissant `top_k=50` :

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k to 50
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog for the rest of the day, but this time it was hard for me to figure out what to do with it. (One reason I asked this for a few months back is that I had a


Pas mal du tout! Le texte est sans doute le texte le plus *à consonance humaine* jusqu'à présent.
Un problème cependant avec l'échantillonnage *Top-K* est qu'il n'adapte pas dynamiquement le nombre de mots filtrés à partir de la distribution de probabilité du mot suivant $P(w|w_{1:t-1})$.
Cela peut être problématique car certains mots peuvent être échantillonnés à partir d'une distribution très nette (distribution à droite dans le graphique ci-dessus), tandis que d'autres peuvent provenir d'une distribution beaucoup plus plate (distribution à gauche dans le graphique ci-dessus).

A l'étape $t=1$, *Top-K* élimine la possibilité de
sample $\text{"people", "big", "house", "cat"}$, qui semblent être des candidats raisonnables. D'un autre côté, à l'étape $t=2$, la méthode inclut les mots sans doute mal adaptés $\text{"down", "a"}$ dans l'échantillon de mots. Ainsi, limiter le pool d'échantillons à une taille fixe *K* pourrait mettre le modèle en danger de produire du charabia pour des distributions nettes et limiter la créativité du modèle pour une distribution plate.
Cette intuition a conduit [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751) pour créer un échantillonnage ***Top-p*** ou ***nucleus***.

### **Échantillonnage Top-p (noyau)**

Au lieu d'échantillonner uniquement à partir des mots *K* les plus probables, dans l'échantillonnage *Top-p*, on choisit parmi le plus petit ensemble possible de mots dont la probabilité cumulée dépasse la probabilité *p*. La masse de probabilité est alors redistribuée parmi cet ensemble de mots. De cette façon, la taille de l'ensemble de mots (*alias* le nombre de mots dans l'ensemble) peut augmenter et diminuer dynamiquement en fonction de la distribution de probabilité du mot suivant. Ok, c'était très verbeux, visualisons.

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

Après avoir défini $p=0,92$, l'échantillonnage *Top-p* sélectionne le nombre *minimum* de mots à dépasser ensemble $p=92\%$ de la masse de probabilité, définie comme $V_{\text{top-p}} $. Dans le premier exemple, cela incluait les 9 mots les plus probables, alors qu'il suffit de sélectionner les 3 premiers mots dans le deuxième exemple pour dépasser 92 %. Assez simple en fait ! On peut voir qu'il conserve un large éventail de mots là où le mot suivant est sans doute moins prévisible, *par exemple* $P(w | \text{"The"})$, et seulement quelques mots lorsque le mot suivant semble plus prévisible, *par exemple* $P(w | \text{"The", "car"})$.

Très bien, il est temps de vérifier cela dans « Transformers » !
Nous activons l'échantillonnage *Top-p* en définissant `0 < top_p < 1` :

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k to 50
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog for the rest of the day, but this had me staying in an unusual room and not going on nights out with friends (which will always be my yearning for such a spacious screen on my desk


Génial, on dirait que cela aurait pu être écrit par un humain. Eh bien, peut-être pas encore.

Alors qu'en théorie, *Top-p* semble plus élégant que *Top-K*, les deux méthodes fonctionnent bien en pratique. *Top-p* peut également être utilisé en combinaison avec *Top-K*, ce qui permet d'éviter les mots très mal classés tout en permettant une sélection dynamique.

Enfin, pour obtenir plusieurs sorties échantillonnées indépendamment, nous pouvons *à nouveau* définir le paramètre `num_return_sequences > 1` :

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3,
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog for the rest of the day, but this time it was hard for me to figure out what to do with it. When I finally looked at this for a few moments, I immediately thought, "
1: I enjoy walking with my cute dog. He has this weird sense of smell. I like walking with him, especially when it comes to walking indoors or if it's a bit chilly. He does walk a lot during the day as well
2: I enjoy walking with my cute dog.

You've done a lot to help the homeless. Now it's time for our next step: help you find them.


Cool, vous devriez maintenant avoir tous les outils pour laisser votre modèle écrire vos histoires avec des « transformateurs » !